In [1]:
import pandas as pd
import tensorflow as tf
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

In [2]:
# Load the data
df = pd.read_csv("./balenced_data.csv")  # Replace with the path to your DataFrame
texts = df['text'].values
labels = df['Source'].values

In [3]:
# Convert the labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [4]:
# Split the data into training and testing sets
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Tokenize the texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_train)

# Convert text sequences to numerical sequences
sequences_train = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)

# Pad the sequences to ensure equal length
max_seq_length = 100 #max(len(seq) for seq in sequences_train)
sequences_train = pad_sequences(sequences_train, maxlen=max_seq_length)
sequences_test = pad_sequences(sequences_test, maxlen=max_seq_length)

In [5]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((sequences_train, labels_train))
test_dataset = tf.data.Dataset.from_tensor_slices((sequences_test, labels_test))

# Apply data pipeline optimizations
train_dataset = train_dataset.shuffle(len(sequences_train)).batch(16).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(16).prefetch(tf.data.AUTOTUNE)

In [6]:
# Define the model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50 #100

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_seq_length))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [7]:
# Compile the models
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# Train the model
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
1169/1169 [==============================] - 171s 145ms/step - loss: 0.7136 - accuracy: 0.7073 - val_loss: 0.2570 - val_accuracy: 0.9181
Epoch 2/10
1169/1169 [==============================] - 168s 144ms/step - loss: 0.2379 - accuracy: 0.9307 - val_loss: 0.2065 - val_accuracy: 0.9285
Epoch 3/10
1169/1169 [==============================] - 168s 143ms/step - loss: 0.1291 - accuracy: 0.9645 - val_loss: 0.2119 - val_accuracy: 0.9373
Epoch 4/10
1169/1169 [==============================] - 168s 144ms/step - loss: 0.0775 - accuracy: 0.9774 - val_loss: 0.2275 - val_accuracy: 0.9377
Epoch 5/10
1169/1169 [==============================] - 168s 144ms/step - loss: 0.0536 - accuracy: 0.9845 - val_loss: 0.3651 - val_accuracy: 0.9191
Epoch 6/10
1169/1169 [==============================] - 168s 144ms/step - loss: 0.0454 - accuracy: 0.9880 - val_loss: 0.3814 - val_accuracy: 0.9014
Epoch 7/10
1169/1169 [==============================] - 168s 144ms/step - loss: 0.0370 - accuracy: 0.9897 - val_

In [9]:
# Perform prediction on the test set
predictions = model.predict(sequences_test)
predicted_labels = label_encoder.inverse_transform(predictions.argmax(axis=1))

# Print the first 100 symbols of the text, true label, and predicted class
print("Text:", texts_test[:100])
print("True Label:", label_encoder.inverse_transform(labels_test[:100]))
print("Predicted Label:", predicted_labels[:100])

147/147 [==============================] - 2s 10ms/step
Text: ['Democratic presidential candidate Senator Bernie Sanders stated that he would “end the deportations that we’re seeing right now” during a town hall on MSNBC on Monday.  Sanders said, in response to a question from a woman who told him that her parents were illegal immigrants, “I believe, absolutely, that we have to move aggressively toward comprehensive immigration reform. My dad was an immigrant. He came to this country at the age of 17. I know a little bit about the immigrant experience. Comprehensive immigration reform, and a path towards citizenship, now, my concern is that if Congress does not do what it should do, and pass that legislation, I will pick up where President Obama left office, and use the executive powers of the presidency to do everything that I can to make your parents safe in this country and not afraid. And the other thing that I will do, where I do disagree with President Obama, I will end the depor

In [13]:
# Convert labels_test back to original form
true_labels = label_encoder.inverse_transform(labels_test)

# Now calculate the metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score

# Compute the metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')
kappa = cohen_kappa_score(true_labels, predicted_labels)

# Print the metrics
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1)
print("Cohen’s Kappa: ", kappa)


Accuracy:  0.9283269148480958
Precision:  0.9287879049726364
Recall:  0.9283269148480958
F1-score:  0.9283315071376751
Cohen’s Kappa:  0.9163620406873655


In [11]:
print(type(labels_test[0]))
print(type(predicted_labels[0]))

<class 'numpy.int32'>
<class 'str'>


In [7]:
# Define the file path for saving / loading the model
model_path = './models/nn.h5'

In [ ]:
# Save the trained model
model.save(model_path)

print("Model saved successfully.")

In [8]:
# Load the saved model
model = load_model(model_path)

print("Model loaded successfully.")

Model loaded successfully.


In [19]:
# Select a random index from the test dataset
random_index = random.randint(0, len(sequences_test) - 1)

# Get the text, true label, and corresponding predicted label at the random index
random_text = texts_test[random_index]
true_label = labels_test[random_index]

# Preprocess the random text
random_sequence = tokenizer.texts_to_sequences([random_text])
random_sequence = pad_sequences(random_sequence, maxlen=max_seq_length)

# Perform prediction on the preprocessed random text
prediction = model.predict(random_sequence)
predicted_label = label_encoder.inverse_transform(prediction.argmax(axis=1))

# Print the first 100 symbols of the text, true label, and predicted class
print("Text:", random_text[:100])
print("True Label:", label_encoder.inverse_transform([true_label])[0])
print("Predicted Label:", predicted_label[0])

1/1 [==============================] - 0s 19ms/step
Text: You guys make me laugh. What do you think? All you need is my number? OK, it's 06 12 23 23 41. You'l
True Label: movie
Predicted Label: blog
